In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
import pickle

ModuleNotFoundError: No module named 'fastai'

In [ ]:
path = Path('/home/jupyter/rauk/training-data')
path.ls()

In [ ]:
path_img = path/'train-images'
path_test=path/'test2-images'

In [ ]:
bs=64

In [ ]:
np.random.seed(42)
data= ImageDataBunch.from_csv(path,folder='train-images',csv_labels='train.csv', valid_pct=0.20, test=path_test, ds_tfms=get_transforms(), size=224, bs=bs
                                  ).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6), ds_type=DatasetType.Valid)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
mapping={}
for k,l in enumerate(data.classes):
    mapping[k]=l


In [ ]:
data.valid_ds.x

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)


In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(2, max_lr=slice(3e-5,3e-4))

In [ ]:
learn.save('stage-2')


In [ ]:
learn.load('stage-2');


In [ ]:
q=learn.TTA(ds_type=DatasetType.Test)

In [ ]:
image=[]
category=[]

In [ ]:
for k in range(0,118863):
    image.append(str(learn.data.test_ds.x.items[k]).split('/')[-1])
    log_pred=q[0][k]
    arr=np.argsort(log_pred)[-3:].numpy()[::-1]
    category.append(str(mapping[arr[0]])+','+str(mapping[arr[1]])+','+str(mapping[arr[2]]))

In [ ]:
res = pd.DataFrame({'Image-Id': image,'Category': category})

In [ ]:
res.to_csv(path/'resulttestfinal.csv', index=False)